# RAPORT SPRZEDAŻY

## Pobieranie danych żródłowych

In [ ]:
# Biblioteki wejściowe
import pandas as pd
import numpy as np
import posixpath as os
import pathlib
from pathlib import Path
import pyodbc
import sys
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from pandas.tseries.offsets import MonthEnd
import IPython

pd.set_option("display.max_rows", None)
#pd.options.display.max_rows=50000
pd.options.display.max_colwidth=70
pd.options.display.precision=2
pd.options.display.max_columns=50
pd.options.display.float_format='{0:,.2f}'.format

#DATA = Path(r'C:\Users\p.przybytniak\Desktop\RAPORTY_ERP_XL\Raport sprzedaży\DATA')
DATA = Path(r'C:\Users\Andrzej.Szajbel\Desktop\RAPORTY_ERP_XL\Raport sprzedaży\DATA')
ASSETS = Path(r'C:\Users\Andrzej.Szajbel\Desktop\RAPORTY_ERP_XL\Raport sprzedaży\ASSETS')
#IMAGES = Path(r'/Users/daniel_wojcik/Documents/KOPALNIA/MIS_REPORT_2022/MONTHLY_REPORTS_2022/MR_202211/IMAGES')


In [ ]:
# ZAPYTANIE SQL -Kontrahenci

t_sql="SELECT  Knt_Akronim as Kontrahent_Id,  ltrim(rtrim(Knt_Nazwa1+Knt_Nazwa2+Knt_Nazwa3)) as Kontrahent_Nazwa , Knt_KodP as Kontrahent_Kod_Pocztowy , Knt_Miasto as Kontrahent_miasto , Knt_NipE as Kontrahent_NIP , Knt_Powiat as Kontrahen_powiat , Knt_Gmina as Kontrahent_Gmina , Knt_Wojewodztwo as Kontrahent_Wojewodztwo , Knt_Modem as Kontrahent_INCO , case Knt_ExpoKraj  	when 1 then 'Krajowy' 	when 2 then 'Unijny' 	else 'Zagraniczny' end As Kontrahent_Status , case Knt_PlatnikVat  	when 1 then 'Czynny VAT' 	else 'Niezarejestrowany VAT' end as Kontrahent_VAT , case when Knt_AkwNumer<>0 then Prc_Akronim else '<Brak>' end as Kontrahent_Handlowiec , CDN.PobierzSciezkeRejonu(KnR_Rejon) AS Kontrahent_Rejon FROM CDN.PrcKarty RIGHT OUTER JOIN      CDN.KntKarty ON Prc_GIDTyp = Knt_AkwTyp AND Prc_GIDNumer = Knt_AkwNumer LEFT OUTER JOIN      CDN.Rejony AS MPK INNER JOIN      CDN.KntRejony ON MPK.REJ_Id = KnR_Rejon INNER JOIN      CDN.Rejony AS Kanal ON MPK.REJ_Id = Kanal.REJ_Id ON Knt_GIDNumer = KnR_KntNumer WHERE  Knt_Akronim LIKE '%_000' "


cnxn=pyodbc.connect('DRIVER={SQL Server};'
                    'Server=192.168.0.120;'
                    'Database=ERPXL_INCO;'
                    'UID=COGS;'
                    'PWD=Biostar_2023;'
                    'Trusted_Connection=yes;')

t_kontrahenci=pd.read_sql_query (t_sql,cnxn)
t_kontrahenci.to_csv(DATA / 'RS_Zapisy_Kontrahent.csv',sep='\t',index=False)

In [ ]:
# ZAPYTANIE SQL -Kartoteka

t_sql="exec [CDN].[PRP_Towary_Atrybuty]"


cnxn=pyodbc.connect('DRIVER={SQL Server};'
                    'Server=192.168.0.120;'
                    'Database=ERPXL_INCO;'
                    'UID=COGS;'
                    'PWD=Biostar_2023;'
                    'Trusted_Connection=yes;')

t_kartoteka=pd.read_sql_query (t_sql,cnxn)
t_kartoteka.fillna(0,inplace=True)
t_kartoteka.to_csv(DATA / 'RS_Zapisy_kartoteka.csv',sep='\t',index=False)

In [ ]:
%%time
t_sql="SELECT  CDN.NazwaObiektu(Dok.TrN_GIDTyp, Dok.TrN_GIDNumer, Dok.TrN_GIDLp, 2) AS DOK_Nr ,Dok.TrN_TrNSeria as DOK_Seria, Dok.TrN_Zaksiegowano as DOK_IFFS , wz.TrN_Zaksiegowano as DOK_IFWZ , dateadd(d,Dok.TrN_Data2,'18001228') as DOK_Data_SP, case Dok.TrN_DataKsiegowania when 0 then null else dateadd(d,Dok.TrN_DataKsiegowania,'18001228') end DOK_Data_FK , Docelowy.Knt_Akronim AS CUST_Odbiorca , Platnik.Knt_Akronim AS CUST_Płatnik , TrE_TwrNumer as PROD_Id , TrE_TwrKod as PROD_Kod , TrE_TwrNazwa as PROD_Nazwa , TrE_StawkaPod as VAL_Vat , TrE_Ilosc as VAL_Ilość , TrE_JmZ as PROD_Jp , TrE_Waluta as VAL_FX_Kod , TrE_KursL/TrE_KursM as VAL_FX_Kurs , TrE_Poczatkowa as VAL_FX_CJ , TrE_Rabat as VAL_PRC_U , TrE_WartoscPoRabacie as VAL_FX_CCU , TrE_KsiegowaNetto as VAL_PLN_CCU , TrE_KosztKsiegowy as COGS_STD , TrE_KosztRzeczywisty as COGS_REAL , Dok.TrN_CechaOpis as DOK_Info_2, Dok.TrN_PrzyczynaKorekty as DOK_Info_1 , TnO_Opis as DOK_Info_3 , dateadd(d,WZ.TrN_DataMag,'18001228') DOK_Data_WZ , cdn.PRFS_Konto_Przychod(TrE_GIDTyp, TrE_GIDNumer, TrE_GIDLp,1) as DOK_Konto , case Dok.TrN_AkwNumer when 0 then '' else '' end SP_PH , case Dok.TrN_OpiNumer when 0 then '' else (select Prc_Akronim from CDN.PrcKarty where Prc_GIDNumer = Dok.TrN_OpiNumer) end SP_KAM FROM CDN.KntKarty AS Platnik INNER JOIN      CDN.TraPlat ON Platnik.Knt_GIDTyp = TrP_KntTyp AND Platnik.Knt_GIDNumer = TrP_KntNumer RIGHT OUTER JOIN      CDN.TraNag AS Dok INNER JOIN      CDN.TraNag AS WZ ON Dok.TrN_GIDTyp = WZ.TrN_SpiTyp AND Dok.TrN_GIDNumer = WZ.TrN_SpiNumer INNER JOIN        CDN.Serie ON Dok.TrN_TrNSeria = Serie.SER_Nazwa INNER JOIN      CDN.KntKarty AS Docelowy ON Dok.TrN_KnDTyp = Docelowy.Knt_GIDTyp AND Dok.TrN_KnDNumer = Docelowy.Knt_GIDNumer ON CDN.TraPlat.TrP_GIDTyp = Dok.TrN_GIDTyp AND       CDN.TraPlat.TrP_GIDNumer = Dok.TrN_GIDNumer LEFT OUTER JOIN      CDN.TraElem ON WZ.TrN_GIDNumer = TrE_GIDNumer LEFT OUTER JOIN      CDN.TrNOpisy ON Dok.TrN_GIDNumer = CDN.TrNOpisy.TnO_TrnNumer WHERE Dok.TrN_GIDTyp IN (2033, 2041, 2037, 2045)" 

cnxn=pyodbc.connect('DRIVER={SQL Server};'
                    'Server=192.168.0.120;'
                    'Database=ERPXL_INCO;'
                    'UID=COGS;'
                    'PWD=Biostar_2023;'
                    'Trusted_Connection=yes;')

t_dane_sprzedaży=pd.read_sql_query (t_sql,cnxn)
t_dane_sprzedaży.to_csv(DATA / 'RS_Zapisy_dane_sprzedaży.csv',sep='\t',index=False)

## Pobieranie danych zapisanych

In [ ]:
%%time
t_kontrahenci=pd.read_csv(DATA /'RS_Zapisy_Kontrahent.csv',sep='\t',low_memory=False)
t_kartoteka=pd.read_csv(DATA /'RS_Zapisy_kartoteka.csv',sep='\t',low_memory=False)
t_dane_sprzedaży=pd.read_csv(DATA /'RS_Zapisy_dane_sprzedaży.csv',sep='\t',low_memory=False)

In [ ]:
t_dane_sprzedaży["VAL_PLN_CC"]=t_dane_sprzedaży["VAL_FX_CJ"]*t_dane_sprzedaży["VAL_FX_Kurs"]*t_dane_sprzedaży['VAL_Ilość']

In [ ]:
t_dane_sprzedaży["VAL_PLN_U"]=t_dane_sprzedaży["VAL_PLN_CC"]-t_dane_sprzedaży["VAL_PLN_CCU"]

In [ ]:
%%time
t_sprzedaż_w=t_dane_sprzedaży[[
    'DOK_Konto',
    'DOK_Nr', 
    'DOK_Seria',
    'DOK_IFFS', 
    'DOK_Data_SP',
    'DOK_Info_1',
    'DOK_Info_2', 
    'CUST_Odbiorca', 
    'CUST_Płatnik', 
    'PROD_Kod', 
    'VAL_Ilość', 
    'VAL_FX_Kod', 
    'VAL_PLN_CC', 
    'VAL_PLN_U', 
    'VAL_PLN_CCU', 
    'SP_PH', 
    'SP_KAM'
]].copy()

In [ ]:
t_sprzedaż_w["DOK_Info_1"].fillna("Faktura pierwotna",inplace=True)

In [ ]:
t_sprzedaż_w=t_sprzedaż_w[t_sprzedaż_w["DOK_Seria"].str.slice(2,3)=="T"]

In [ ]:
t_sprzedaż_w['DOK_K_E']=t_sprzedaż_w['DOK_Seria'].str.slice(1,2)
t_sprzedaż_w['DOK_K_E']=t_sprzedaż_w.apply(lambda x:'Kraj' if x['DOK_K_E']=="S" else 'Eksport', axis=1)

In [ ]:
t_sprzedaż_w_kor=t_sprzedaż_w[t_sprzedaż_w["DOK_Info_1"]!="Faktura pierwotna"].copy()

In [ ]:
t_sprzedaż_w=t_sprzedaż_w[t_sprzedaż_w["DOK_Info_1"]=="Faktura pierwotna"]

In [ ]:
t_kartoteka_draft=t_kartoteka[[
    'Kod',
    'Nazwa',
    'Drzewko',
    'Rodzaj',
    'Grupa',
    'Typ_',
    'Marka',
    'Etykieta',
    'Kategoria',
    'Promocja',
    'Waga_netto(JM)'
]].copy()

In [ ]:
t_kartoteka_draft['Drzewko']=t_kartoteka_draft['Drzewko'].str.split('/').str[1]

In [ ]:
t_kartoteka_draft.rename(columns={'Kod':'PROD_Kod','Nazwa':'PROD_Nazwa','Drzewko':'PROD_SOD','Rodzaj':'PROD_Rodzaj','Grupa':'PROD_Grupa','Typ_':'PROD_Typ','Marka':'PROD_Marka','Etykieta':'PROD_Etykieta','Kategoria':'PROD_Kategoria','Promocja':'PROD_Promocja','Waga_netto(JM)':'PROD_Waga'}, inplace=True)

In [ ]:
t_sprzedaż_w=pd.merge(t_sprzedaż_w,t_kartoteka_draft,on='PROD_Kod',how='left')

In [ ]:
t_kontrahenci.rename(columns={'Kontrahent_Id':'CUST_Płatnik','Kontrahent_Nazwa':'CUST_Nazwa','Kontrahent_Rejon':'CUST_Kanał_temp'}, inplace=True)

In [ ]:
t_kontrahent=t_kontrahenci[['CUST_Płatnik','CUST_Nazwa','CUST_Kanał_temp']].copy()

In [ ]:
t_kontrahent['CUST_Kanał']=t_kontrahent['CUST_Kanał_temp'].str.split('\\').str[0]

In [ ]:
t_kontrahent['CUST_PH']=t_kontrahent['CUST_Kanał_temp'].str.split('\\').str[-1]

In [ ]:
t_kontrahent['CUST_Kanał'].fillna('POZ', inplace=True)
t_kontrahent['CUST_PH'].fillna('POZ', inplace=True)

t_kontrahent.drop(['CUST_Kanał_temp'],axis=1,inplace=True)

In [ ]:
# Zapisywanie kartoteki do excel
t_kontrahenci.to_excel(os.join(DATA,r'\kontrahenci.xlsx'),index=False)

In [ ]:
t_sprzedaż_w=pd.merge(t_sprzedaż_w,t_kontrahent,how='left',on=['CUST_Płatnik'])

### Aktualizacja wyjątków

In [ ]:
# Aktualizuje Biedronkę
t_sprzedaż_w['CUST_Kanał']=t_sprzedaż_w.apply(lambda x:'DSS' if (x['CUST_Płatnik']=='K200003_000' and x['PROD_Marka']=='LUDWIK') else x['CUST_Kanał'],axis=1)

In [ ]:
# Aktualizacja Marek Własnych
t_sprzedaż_w['CUST_Kanał']=t_sprzedaż_w.apply(lambda x:'WS' if ((x['CUST_Kanał']!='POZ' and x['CUST_Kanał']!='WSE') and 
                                                                   (x['PROD_Kod'].startswith('CG106')==True or
                                                                    x['PROD_Kod'].startswith('CG108')==True or
                                                                    x['PROD_Kod'].startswith('CGP06')==True or
                                                                    x['PROD_Kod'].startswith('CGP08')==True or
                                                                    x['PROD_Kod'].startswith('NW106')==True or
                                                                    x['PROD_Kod'].startswith('NWP06')==True or
                                                                    x['PROD_Kod'].startswith('PL106')==True or
                                                                    x['PROD_Kod'].startswith('PLP06')==True)) else x['CUST_Kanał'],axis=1)

In [ ]:
# Aktualizacja DSNP
t_sprzedaż_w['CUST_Kanał']=t_sprzedaż_w.apply(lambda x:'DSNP' if (x['PROD_Kod'].startswith('NW108')==True and x['PROD_Kod']!='NW1080201N01AG011' and x['CUST_Kanał']=='DSNH') else x['CUST_Kanał'],axis=1)
t_sprzedaż_w['CUST_Kanał']=t_sprzedaż_w.apply(lambda x:'DSNP' if (x['PROD_Kod'].startswith('NW108')==True and x['PROD_Kod']!='NW1080201N01AG011' and x['CUST_Kanał']=='DSNS') else x['CUST_Kanał'],axis=1)
t_sprzedaż_w['CUST_Kanał']=t_sprzedaż_w.apply(lambda x:'DSNH' if (x['PROD_Kod']=='NW1080201N01AG011' and x['CUST_Kanał']=='DSNP') else x['CUST_Kanał'],axis=1)

t_sprzedaż_w['CUST_Kanał']=t_sprzedaż_w.apply(lambda x:'DSNP' if ((x['PROD_Kod'].startswith('NW1060206')==True or x['PROD_Kod'].startswith('NW1060207')==True) and x['CUST_Kanał']!='POZ') else x['CUST_Kanał'],axis=1)
t_sprzedaż_w['CUST_Kanał']=t_sprzedaż_w.apply(lambda x:'DSNP' if (x['PROD_Kod'].startswith('NW1020201N01AI011')==True and x['CUST_Kanał']=='DSNH') else x['CUST_Kanał'],axis=1)

t_sprzedaż_w['CUST_Kanał']=t_sprzedaż_w.apply(lambda x:'DSNP' if 
                                              (x['PROD_Kod']=='NW1020201N01AI011' and x['CUST_Kanał']=='DSNH')
                                              or (x['PROD_Kod']=='NW1060101N97BA011' and x['CUST_Kanał']=='WS')
                                              or (x['PROD_Kod']=='NW1010101N01AK011' and x['CUST_Kanał']=='DSNH')
                                              or (x['PROD_Kod']=='NW1050102N02AG011' and x['CUST_Kanał']=='DSNH')
                                              or (x['PROD_Kod']=='NW1060114N90AK011' and x['CUST_Kanał']=='WS')
                                              else x['CUST_Kanał'],axis=1)

In [ ]:
# Aktualizacja GCG DSNS
t_sprzedaż_w['CUST_Kanał']=t_sprzedaż_w.apply(lambda x:'DSS' if (x['PROD_Kod'].startswith('CG')==True and x['CUST_Kanał']=='DSNS') else x['CUST_Kanał'],axis=1)
t_sprzedaż_w['CUST_Kanał']=t_sprzedaż_w.apply(lambda x:'DSS' if (x['PROD_Kod'].startswith('PL')==True and x['CUST_Kanał']=='DSNS') else x['CUST_Kanał'],axis=1)
t_sprzedaż_w['CUST_Kanał']=t_sprzedaż_w.apply(lambda x:'DSS' if (x['PROD_Kod'].startswith('NW')==True and x['CUST_Kanał']=='DSS') else x['CUST_Kanał'],axis=1)

In [ ]:
# Aktualizacja Biofos DSNS
t_sprzedaż_w['CUST_Kanał']=t_sprzedaż_w.apply(lambda x:'DSNS' if (
    (x['PROD_Kod'].startswith('CG10509')==True or x['PROD_Kod'].startswith('CGP0509')==True) and 
    (x['CUST_Kanał']=='DSNS') and
    (x['CUST_Płatnik']=='K200035_000' or x['CUST_Płatnik']=='K200033_000')) else x['CUST_Kanał'],axis=1)

In [ ]:
# Aktualizuje Kanał KGO Zewnętrzna
t_sprzedaż_w['CUST_Kanał']=t_sprzedaż_w.apply(lambda x:'KGO_Z' if (x['CUST_Kanał'] =='KGO') else x['CUST_Kanał'], axis=1)

In [ ]:
t_sprzedaż_w.head()

In [27]:
# Aktualizacja DSNP
t_sprzedaż_w['CUST_Kanał']=t_sprzedaż_w.apply(lambda x:'DSNP' if (x['PROD_Kod'].startswith('NW108')==True and x['PROD_Kod']!='NW1080201N01AG011' and x['CUST_Kanał']=='DSNH') else x['CUST_Kanał'],axis=1)
t_sprzedaż_w['CUST_Kanał']=t_sprzedaż_w.apply(lambda x:'DSNP' if (x['PROD_Kod'].startswith('NW108')==True and x['PROD_Kod']!='NW1080201N01AG011' and x['CUST_Kanał']=='DSNS') else x['CUST_Kanał'],axis=1)
t_sprzedaż_w['CUST_Kanał']=t_sprzedaż_w.apply(lambda x:'DSNH' if (x['PROD_Kod']=='NW1080201N01AG011' and x['CUST_Kanał']=='DSNP') else x['CUST_Kanał'],axis=1)

t_sprzedaż_w['CUST_Kanał']=t_sprzedaż_w.apply(lambda x:'DSNP' if ((x['PROD_Kod'].startswith('NW1060206')==True or x['PROD_Kod'].startswith('NW1060207')==True) and x['CUST_Kanał']!='POZ') else x['CUST_Kanał'],axis=1)
t_sprzedaż_w['CUST_Kanał']=t_sprzedaż_w.apply(lambda x:'DSNP' if (x['PROD_Kod'].startswith('NW1020201N01AI011')==True and x['CUST_Kanał']=='DSNH') else x['CUST_Kanał'],axis=1)

t_sprzedaż_w['CUST_Kanał']=t_sprzedaż_w.apply(lambda x:'DSNP' if 
                                              (x['PROD_Kod']=='NW1020201N01AI011' and x['CUST_Kanał']=='DSNH')
                                              or (x['PROD_Kod']=='NW1060101N97BA011' and x['CUST_Kanał']=='WS')
                                              or (x['PROD_Kod']=='NW1010101N01AK011' and x['CUST_Kanał']=='DSNH')
                                              or (x['PROD_Kod']=='NW1050102N02AG011' and x['CUST_Kanał']=='DSNH')
                                              or (x['PROD_Kod']=='NW1060114N90AK011' and x['CUST_Kanał']=='WS')
                                              else x['CUST_Kanał'],axis=1)

In [28]:
# Aktualizacja GCG DSNS
t_sprzedaż_w['CUST_Kanał']=t_sprzedaż_w.apply(lambda x:'DSS' if (x['PROD_Kod'].startswith('CG')==True and x['CUST_Kanał']=='DSNS') else x['CUST_Kanał'],axis=1)
t_sprzedaż_w['CUST_Kanał']=t_sprzedaż_w.apply(lambda x:'DSS' if (x['PROD_Kod'].startswith('PL')==True and x['CUST_Kanał']=='DSNS') else x['CUST_Kanał'],axis=1)
t_sprzedaż_w['CUST_Kanał']=t_sprzedaż_w.apply(lambda x:'DSS' if (x['PROD_Kod'].startswith('NW')==True and x['CUST_Kanał']=='DSS') else x['CUST_Kanał'],axis=1)

In [29]:
# Aktualizacja Biofos DSNS
t_sprzedaż_w['CUST_Kanał']=t_sprzedaż_w.apply(lambda x:'DSNS' if (
    (x['PROD_Kod'].startswith('CG10509')==True or x['PROD_Kod'].startswith('CGP0509')==True) and 
    (x['CUST_Kanał']=='DSNS') and
    (x['CUST_Płatnik']=='K200035_000' or x['CUST_Płatnik']=='K200033_000')) else x['CUST_Kanał'],axis=1)

In [30]:
# Aktualizuje Kanał KGO Zewnętrzna
t_sprzedaż_w['CUST_Kanał']=t_sprzedaż_w.apply(lambda x:'KGO_Z' if (x['CUST_Kanał'] =='KGO') else x['CUST_Kanał'], axis=1)

In [31]:
t_sprzedaż_w.head()

,DOK_Konto,DOK_Nr,DOK_Seria,DOK_IFFS,DOK_Data_SP,DOK_Info_1,DOK_Info_2,CUST_Odbiorca,CUST_Płatnik,PROD_Kod,VAL_Ilość,VAL_FX_Kod,VAL_PLN_CC,VAL_PLN_U,VAL_PLN_CCU,SP_PH,SP_KAM,DOK_K_E,PROD_Nazwa,PROD_SOD,PROD_Rodzaj,PROD_Grupa,PROD_Typ,PROD_Marka,PROD_Etykieta,PROD_Kategoria,PROD_Promocja,PROD_Waga,CUST_Nazwa,CUST_Kanał,CUST_PH
0,701-02-01-01-100-02,(S)FS-GSTFP/23/01/1,GSTFP,0,2023-01-05,Faktura pierwotna,SAGRA,K200019_000,K200019_000,CG1050962Z0010121,12.00,PLN,101.64,16.32,85.32,NaN,NaN,Kraj,BIOFOS Wc żel BIO 500ml a.12,GCG,Wyroby gotowe,Pozostałe,Żel do WC,BIOFOS,EKO,POZOSTAŁE,Prod. Standardowe,"0,5225",KOMPLEX-TORUS SP. Z O.O.,DSH,HWA
1,701-02-01-01-100-02,(S)FS-GSTFP/23/01/1,GSTFP,0,2023-01-05,Faktura pierwotna,SAGRA,K200019_000,K200019_000,CGP050902M0110041,8.00,PLN,302.32,48.40,253.92,NaN,NaN,Kraj,"Zestaw Biofos Professional (proszek 1,0 kg + żel WC BIO 500 ml) a.4",GCG,Wyroby gotowe,Pozostałe,Do szamb,BIOFOS,Standard,POZOSTAŁE,Prod. Standardowe,"1,5275",KOMPLEX-TORUS SP. Z O.O.,DSH,HWA
2,701-02-01-01-100-03,FS-ISTFP/23/01/1,ISTFP,0,2023-01-05,Faktura pierwotna,NaN,K100001_000,K100001_000,NW1020102N77D0011,1.00,PLN,52.54,3.89,48.65,NaN,NaN,Kraj,"FLOROVIT NAWÓZ DO TRAWNIKÓW SZYBKI EFEKT 5,0 KG",GNW,Wyroby gotowe,Nawozy do trawników,Granulat,FLOROVIT,Standard,POZOSTAŁE,Prod. Standardowe,5,"""GRUPA INCO"" SPÓŁKA AKCYJNA",POZ,POZ
3,XXX-02-01-99-100-01,FS-ISTFP/23/01/1,ISTFP,0,2023-01-05,Faktura pierwotna,NaN,K100001_000,K100001_000,PRZESYŁKA8,1.00,PLN,12.99,0.96,12.03,NaN,NaN,Kraj,PRZESYŁKA8,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,"""GRUPA INCO"" SPÓŁKA AKCYJNA",POZ,POZ
4,701-02-01-01-100-02,FS-ISTFP/23/01/2,ISTFP,0,2023-01-09,Faktura pierwotna,NaN,K204013_000,K204013_000,CG1010113Z0210121,24.00,PLN,157.68,29.48,128.20,NaN,NaN,Kraj,Ludwik płyn do mycia naczyń cytrynowy 900 g a'12 K,GCG,Wyroby gotowe,Płyn do mycia naczyń,"od 0,90 do 1,34 [kg:l]",LUDWIK,Standard,POZOSTAŁE,Prod. Standardowe,"0,9",Uniwersytet Przyrodniczy w Lublinie,POZ,POZ
